### Import Required Libraries and Set Up Environment Variables

In [ ]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [ ]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [ ]:
# Set the base URL
base_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'
# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"
# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"
# Build URL
query_url = f"{base_url}api-key={nyt_api_key}&fq={filter_query}&begin_date={begin_date}&end_date={end_date}&sort={sort}&fl={field_list}"

In [ ]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    url = f"{query_url}&page={page}"

    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(url).json()

    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)

    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f"Checked page number {page}.")
    except:
        # Print the page number that had no results then break from the loop
        print(f"No reviews retrieved for page number {page}.")
        break

In [ ]:
#Confirming 200 reviews were retrieved
print(len(reviews_list))

# Preview the first 5 results in JSON format
first_five = reviews_list[:5]
# Use json.dumps with argument indent=4 to format data
json_data = json.dumps(first_five, indent=4)
print(json_data)

In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
# Preview the first few rows of the DataFrame
reviews_df.head() 

In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019.
# End string should include " Review" to avoid cutting title early
reviews_df["title"] = reviews_df["headline.main"].apply(
    lambda st: st[st.find("\u2018") + 1 : st.find("\u2019 Review")]
)
print(reviews_df["title"])

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keywords = f"{item['name']}: {item['value']};"
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keywords
    return extracted_keywords

In [ ]:
# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"] = reviews_df["keywords"].apply(extract_keywords)
reviews_df.head()

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database in part 2
titles_list = reviews_df["title"].to_list()

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the results
tmdb_movies_list = []   

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles_list:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        # Sleep for a random time between 1 and 5 seconds
        time.sleep(1)
        print("Sleeping for one second")

    # Add 1 to the request counter
    request_counter += 1    

    # Perform a "GET" request for The Movie Database
    tmdb_response = requests.get(url + title + tmdb_key_string) .json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = tmdb_response["results"][0]["id"]

        # Make a request for a the full movie details
        movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"

        # Execute "GET" request with url
        tmdb_response_details = requests.get(movie_details_url).json()

        # Extract the genre names into a list
        genres = [tmdb_response_details['genres'][i]['name'] for i in range(len(tmdb_response_details['genres']))]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [tmdb_response_details['spoken_languages'][i]['english_name'] for i in range(len(tmdb_response_details['spoken_languages']))]

        # Extract the production_countries' name into a list
        production_countries = [tmdb_response_details['production_countries'][i]['name'] for i in range(len(tmdb_response_details['production_countries']))]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        dict_movie_details = {'title': title,
                              'original_title': tmdb_response_details['original_title'],
                              'budget': tmdb_response_details['budget'],
                              'original_language': tmdb_response_details['original_language'],
                              'homepage': tmdb_response_details['homepage'],
                              'overview': tmdb_response_details['overview'],
                              'popularity': tmdb_response_details['popularity'],
                              'runtime': tmdb_response_details['runtime'],
                              'revenue': tmdb_response_details['revenue'],
                              'release_date': tmdb_response_details['release_date'],
                              'vote_average': tmdb_response_details['vote_average'],
                              'vote_count': tmdb_response_details['vote_count'],
                              'genres': genres,
                              'spoken_languages': spoken_languages,
                              'production_countries': production_countries}
        tmdb_movies_list.append(dict_movie_details)

        # Print out the title that was found
        print(f'Title found: "{title}".')
    except Exception as e: 
        print(f"Not found: {title}, error: {e}") 

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

In [ ]:
# Convert the results to a DataFrame
tmdb_movies_df = pd.DataFrame(tmdb_movies_list)
tmdb_movies_df.head(10)

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(tmdb_movies_df, reviews_df, on="title")
display(merged_df.head(10))

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries',]  

# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for col in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[col] = merged_df[col].astype(str)

    # Loop through characters to remove
    for char in characters_to_remove:
        # Remove the character
        merged_df[col] = merged_df[col].str.replace(char, '')

# Display the fixed columns in the  DataFrame, the display is truncated the columns. 
display(merged_df)

In [ ]:
# Display the fixed columns in the  DataFrame
merged_df[["genres", "spoken_languages", "production_countries"]].head(10)

In [ ]:
# Drop "byline.person" column
merged_df.drop("byline.person", axis=1, inplace=True)

In [ ]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates(inplace=True)
merged_df.reset_index(drop=True, inplace=True)
print(merged_df.shape)

In [ ]:
# Export data to CSV without the index
merged_df.to_csv('output/movie_data.csv', index=False)